In [ ]:
import mne
#%gui qt
#import matplotlib

#%matplotlib qt
from nilearn.plotting import plot_stat_map
from nilearn.image import index_img
import numpy as np
from matplotlib import pyplot as plt
from stormdb.access import Query
from pickle import load
from scipy import stats as spstats
from mne.datasets import sample
from mne.stats import spatio_temporal_cluster_1samp_test
import os
import pickle
import pandas as pd
from copy import deepcopy
import warnings
import os.path as op
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# os.environ['ETS_TOOLKIT'] = 'qt4'
# os.environ['QT_API'] = 'pyqt5'
# %gui qt
#mne.viz.set_3d_backend("notebook")

In [ ]:
proj_name = 'MINDLAB2020_MEG-AuditoryPatternRecognition'
wdir = '/projects/' + proj_name + '/scratch/working_memory/'
stats_dir = wdir + 'results/stats/'
data_dir = wdir + 'averages/data/'
figures_dir = wdir + 'results/figures/'

subs_dir = '/projects/' + proj_name + '/scratch/fs_subjects_dir/'
sample_path = sample.data_path()
sample_subjects_dir = sample_path + '/subjects'
src_sample = mne.read_source_spaces(subs_dir +
                                '/fsaverage/bem/fsaverage-vol-5-src.fif')
# src_sample = mne.read_source_spaces(subs_dir +
#                                 'fsaverage/bem/fsaverage_ico4_vol-src.fif')
#label_file = subs_dir + 'fsaverage/mri/aparc.DKTatlas+aseg.mgz'
#label_file = sample_subjects_dir + '/fsaverage/mri/aparc.a2009s+aseg.mgz'
label_file = sample_subjects_dir + '/fsaverage/mri/aparc+aseg.mgz'

In [ ]:
# Read stats
periods = {#'all':[0, 4],
           #'list_vs_im': [0,4],
           'L1': [.2,.5],
           'L2': [.7,1.],
           'L3': [1.2,1.5],
           'I1': [2,4],
           'I2': [2.5,3.5]
           }#,'retrieval': [4,6.25]}{'all': [0.,4]}#
stats = {}

for p in periods:
    csfname = '{}patterns_source_stats_{}_peaks.p'.format(stats_dir, p)
    #csfname = '{}patterns_source_stats_{}.p'.format(stats_dir, p)
    cfile = open(csfname,'rb')
    stats[p] = pickle.load(cfile)
    cfile.close()
print(stats)

In [ ]:
for c in stats:
    print(c)
    for s in stats[c]:
        print(s)

In [ ]:
# Load and morph a source time course
dfname = data_dir + '0021_LZW/0021_LZW_patterns_sources_task_sensor_lf_0.05_hf_None_tstep_0.025_twin_0.05_localized.p'
dfile = open(dfname,'rb')
all_stc = pickle.load(dfile)
dfile.close()
stc = deepcopy(all_stc['maintenance1'])
del all_stc

# load source morph
morph = mne.read_source_morph(subs_dir + '0021_LZW/bem/0021_LZW_vol-morph.h5')
stc = morph.apply(stc)
print(stc)

In [ ]:
labels = mne.get_volume_labels_from_aseg(label_file)

In [ ]:
print(labels)

In [ ]:
#labels = mne.read_labels_from_annot('fsaverage','aparc.a2009s')

In [ ]:
labels

In [ ]:
clabels = ['Left-Thalamus-Proper',
           'Right-Thalamus-Proper',
           'Left-VentralDC',
           'Right-VentralDC',
           'Right-Hippocampus',
           'Left-Hippocampus',
           'ctx_lh_G_oc-temp_med-Parahip',
           'ctx_rh_G_oc-temp_med-Parahip',
           'ctx_lh_G_and_S_cingul-Ant',
           'ctx_rh_G_and_S_cingul-Ant',
           'ctx_lh_G_and_S_cingul-Mid-Ant',
           'ctx_rh_G_and_S_cingul-Mid-Ant',
           'ctx_lh_G_and_S_cingul-Mid-Post',
           'ctx_rh_G_and_S_cingul-Mid-Post',
           'ctx_lh_G_temp_sup-Plan_tempo',
           'ctx_rh_G_temp_sup-Plan_tempo',
           'ctx_lh_G_temp_sup-Lateral',
           'ctx_rh_G_temp_sup-Lateral',
           'Left-Pallidum',
           'Right-Pallidum',
           'Left-Caudate',
           'Right-Caudate',
           'ctx_lh_S_orbital-H_Shaped',
           'ctx_rh_S_orbital-H_Shaped',
           'ctx_lh_G_front_inf-Opercular',
           'ctx_rh_G_front_inf-Opercular',
           'ctx_lh_G_front_inf-Orbital',
           'ctx_rh_G_front_inf-Orbital',
           'ctx_lh_G_front_inf-Triangul',
           'ctx_rh_G_front_inf-Triangul',
           'ctx_lh_G_front_middle',
           'ctx_rh_G_front_middle',
           'ctx_lh_G_front_sup',
           'ctx_rh_G_front_sup',
           'ctx_lh_Lat_Fis-ant-Horizont',
           'ctx_rh_Lat_Fis-ant-Horizont']

stc_labels = {}
se_labels = {}
stc_mask = {}
conds = ['maintenance1','manipulation1','interaction']
for c in conds:
    cdata = stc.copy().crop(-.05,4)
    cmask = stc.copy().crop(-.05,4)
    cse = stc.copy().crop(-.05,4)
    emask = stats['encoding'][c]['mask'].copy()
    dmask = stats['delay'][c]['mask'].copy()

    cdata.data = np.hstack((stats['encoding'][c]['data_mean'],stats['delay'][c]['data_mean'])) # * stats_results[c]['mask']
    cmask.data = np.hstack((emask,dmask))
    print(cmask.data)
    cse.data = np.hstack((stats['encoding'][c]['data_sd'],stats['delay'][c]['data_sd'])) / np.sqrt(stats['encoding'][c]['n'])# * stats_results[c]['mask'] 
#    cdata.data = stats['encoding'][c]['data_mean'].T # * stats_results[c]['mask'] 
#    cse.data = stats['encoding'][c]['data_sd'].T / np.sqrt(stats['encoding'][c]['n'])# * stats_results[c]['mask'] 
    stc_labels[c] = cdata.extract_label_time_course(labels = [label_file,clabels], src = src_sample, mode = 'auto')
    stc_mask[c] = cmask.extract_label_time_course(labels = [label_file,clabels], src = src_sample, mode = 'auto')
    se_labels[c] = cse.extract_label_time_course(labels = [label_file,clabels], src = src_sample, mode = 'auto')

In [ ]:
times = stc.copy().crop(-.025,4).times 
periods = {'encoding': [0,2], 'delay': [2.0,3.975]}
blocks = ['maintenance1','manipulation1']
cur_labs = ['Right Planum Temporale','Right Hippocampus','Right Pallidum',
            'Left Thalamus','Right Ventral Diencephalon',
            'Right Caudate','Right Orbital Sulcus','Anterior Lateral Fissure',
            'Posterior Cingulate']
cur_labs_codes = ['ctx_rh_G_temp_sup-Plan_tempo','Right-Hippocampus','Right-Pallidum',
                  'Left-Thalamus-Proper','Right-VentralDC',
                  'Right-Caudate','ctx_rh_S_orbital-H_Shaped','ctx_lh_Lat_Fis-ant-Horizont',
                  'ctx_rh_G_and_S_cingul-Mid-Post']
lab_ix = []
for ll in cur_labs_codes:
    lab_ix += [np.where(np.array(clabels) == ll)[0][0]]
    print(lab_ix)
#lab_ix = [11,15]
titles = ['maintenance','manipulation']
sidx = -1
fig, axes = plt.subplots(ncols=2,nrows=len(cur_labs), figsize = (12.5,len(cur_labs)*5))

for clix, cl in enumerate(cur_labs):
    for bidx,b in enumerate(blocks):
        lidx = lab_ix[clix]
        sidx = sidx + 1
        rix, cix = sidx//2,sidx%2
        #axes[rix, cix].plot(times,stc_labels[b + '/' + 'mel1'][lidx],color='b')
        #axes[rix, cix].plot(times,stc_labels[b + '/' + 'mel2'][lidx],color='r')
        ci_upper = stc_labels[b][lidx] + 1.96*se_labels[b][lidx]
        ci_lower = stc_labels[b][lidx] - 1.96*se_labels[b][lidx]
        ccmask = np.array(stc_mask[b][lidx] != 0).astype(float)
        ccmask[ccmask==0.] = np.nan
        #axes[rix, cix].fill_between(times, ci_lower, ci_upper, color='k', alpha=.2)
        axes[rix, cix].plot(times,stc_labels[b][lidx],color='k')
        axes[rix, cix].plot(times,stc_labels[b][lidx]*ccmask,color='k',linewidth=4)
        axes[rix, cix].set_title(titles[bidx] + ' ' + cl)
        axes[rix, cix].set_ylim(-3,3)
        axes[rix, cix].set_xlim(-.1,4)
        axes[rix, cix].set_xlabel('time (s)')
        axes[rix, cix].set_ylabel('source activation (a.u.)')
        axes[rix, cix].axhline(0., color='k')
        axes[rix, cix].axvline(0., color='k')
        axes[rix, cix].axvline(2., color='k',linestyle='--')
        axes[rix, cix].axvline(.5, color='k',linestyle=':')
        axes[rix, cix].axvline(1, color='k',linestyle=':')
        #axes[rix, cix].legend([clabels[cls] for cls in [0,8]])
        #axes[rix, cix].legend(['patterns'])#,'difference'])
plt.tight_layout()
plt.savefig(figures_dir + 'patterns_mels_sources_labels_diff.pdf',orientation='portrait')

In [ ]:
def get_cluster_peaks(cstat,p_threshold=.05):
        sigix = np.where(cstat['pvals']<=p_threshold)[0]
        vx, vy, masks, p = [], [], [], []
        for six in sigix:
            cmask = cstat['clusters'][six].copy().astype(int).T
            cdata = cstat['tvals']*cmask
            cvx, cvy = np.unravel_index(np.abs(cdata).argmax(), cdata.shape)
            vx += [cvx]
            vy += [cvy]
            masks += [cmask]
            p += [cstat['pvals'][six]]
        vx = np.array(vx)
        vy = np.array(vy)
        return vx, vy, masks, p

In [ ]:
for pix, pval in enumerate(stats[p][c]['pvals']):
    print(pval)
    print(np.min(np.abs(stats[p][c]['tvals']))*stats[p][c]['clusters'][pix].astype(int))


In [ ]:
conds = ['maintenance1','manipulation1','interaction']#'maintenance1','manipulation1',]
cnames = ['recall','manipulation','difference']
fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(10,10))
for p in stats:
    for cix,c in enumerate(conds):#stats[p]:
        vx, vy, masks, _ = get_cluster_peaks(stats[p][c])
        for v in range(vx.shape[0]):
            cdata = stats[p][c]['data_mean']*masks[v]
            cond_name = cnames[cix]
            cluster_name = p + ' (' + cond_name + ' - cluster ' + str(v+1) + ')'
            ch = ax.plot(stc.copy().crop(tmin=periods[p][0],tmax=periods[p][1]).times,
                     cdata.mean(axis=0),label=cluster_name)
            plt.ylim(-.2,.2)
            plt.xlim(-.1,4)
            plt.xlabel('time (s)')
            plt.ylabel('source activation (a.u.)')
            plt.axhline(0., color='k')
            plt.axvline(0., color='k')
            plt.axvline(2., color='k',linestyle='--')
            plt.axvline(.5, color='k',linestyle=':')
            plt.axvline(1, color='k',linestyle=':')
            plt.legend()
plt.tight_layout()
#plt.savefig(figures_dir + 'patterns_mels_sources_cluster.pdf',orientation='portrait')

In [ ]:
#periods = {'encoding': [0,2], 'delay': [2.0,3.975]}
#periods = {'L2': [1.3,1]}#, 'delay': [2.0,3.975]}
cperiods = ['L1','L2','L3','I1']#,'I2']#'list_vs_im'
conds = ['maintenance1','manipulation1','interaction']
label_sel = [l for l in labels if ('White' not in l) & ('choroid' not in l) & ('vessel' not in l) & ('callosum' not in l) &
              ('Vent' not in l) & ('wall' not in l) & ('Unknown' not in l) & ('unknown' not in l) & ('Cerebellum' not in l)
              & ('CC_' not in l) & ('CSF' not in l) & ('Optic-Chiasm' not in l) & ('WM-hypointensities' not in l)]
MA = {}
clust_report = {'period': [], 'condition': [], 'cluster No': [],
                 'x': [], 'y': [], 'z': [],'p-value': [],
                 'peak t-value': [], 'peak activation': []} 
for p in cperiods:
    print(p)
    MA[p] = {}
    conds = stats[p].keys()
    for c in conds:#stats[p]: 
        print(c)
        cstc = stc.copy().crop(tmin=periods[p][0],tmax=periods[p][1])
        vx, vy, masks, pvals = get_cluster_peaks(stats[p][c])
        MA[p][c] = np.zeros((vx.shape[0],len(label_sel)))
        for v in range(vx.shape[0]):
            ctime = cstc.times[vy[v]]
            print('cluster',v+1,' - peak: ', np.round(ctime,3), 's')
            print(np.unique(masks[v]))
            cstc.data = stats[p][c]['tvals'].copy()*masks[v]
            csign = np.sign(cstc.data.mean())
            cstc.data = np.abs(cstc.data)
            pos, _ = cstc.get_peak()
            pix, _ = cstc.get_peak(vert_as_index=True)
            coords = np.round(src_sample[0]['rr'][pos]*1000)
            clust_report['period'] += [p]
            clust_report['condition'] += [c]
            clust_report['cluster No'] += [v+1]
            clust_report['x'] += [coords[0]]
            clust_report['y'] += [coords[1]]
            clust_report['z'] += [coords[2]]
            clust_report['p-value'] += [pvals[v]]
            clust_report['peak t-value'] += [np.round(cstc.data[pix,0]*csign,2)]
            clust_report['peak activation'] += [np.round(stats[p][c]['data_mean'][pix,0],2)]

            # MA[p][c][v] = np.squeeze(mne.extract_label_time_course(cstc,[label_file,label_sel],
            #                                                         src=src_sample,mode='max'))*csign
clust_report = pd.DataFrame(clust_report)         
print(clust_report)
clust_report.to_csv(op.join(stats_dir,'patterns_source_stats_report.csv'),index=False)

            #MA[p][c][v] = np.max(np.abs(mne.extract_label_time_course(cstc,[label_file,labels],src=src_sample)),axis=1)*np.sign(cstc.data.mean())#.mean(axis=1)

In [ ]:
hems = {'right': ['rh','Right'],'left': ['lh','Left']}
MA_df = {'cluster': [], 'label': [], 'hemisphere': [], 'tval': []}
for p in MA:
    for c in MA[p]:#stats[p]:
        for cl in range(MA[p][c].shape[0]):
            act_ix = np.argsort(-np.abs(MA[p][c][cl,:]))
            #print( '\n', p, c,' cluster ', cl + 1)
            for h in hems:
                #print(h)
                for ix in act_ix:
                    if (np.abs(MA[p][c][cl,ix]) >= 2) & (hems[h][0] in label_sel[ix] or hems[h][1] in label_sel[ix]): 
                        print(label_sel[ix],np.round(MA[p][c][cl,ix],2))
                        MA_df['cluster'] += ['{} {} cluster {}'.format(p, c, cl + 1)]
                        MA_df['label'] += [label_sel[ix]]
                        MA_df['hemisphere'] += [h]
                        MA_df['tval'] += [np.round(MA[p][c][cl,ix],2)]
                             
MA_df = pd.DataFrame(MA_df)
print(MA_df)     
MA_df.to_csv(op.join(stats_dir,'patterns_stats_report_aparc.csv'),index=False)


In [ ]:
conds = ['maintenance1','manipulation1','interaction']#'maintenance1','manipulation1',]
for p in stats:
    for c in conds:#stats[p]:
        
        cstc = stc.copy().crop(tmin=periods[p][0],tmax=periods[p][1])
        vx, vy, masks, _ = get_cluster_peaks(stats[p][c])

        for v in range(vy.shape[0]):
            cstc.data = stats[p][c]['data_mean']*masks[v]
            ctime = cstc.times[vy[v]]
            #cstc = cstc.copy().crop(tmin=ctime-.1,tmax=ctime+.1).mean()
            cstc = cstc.copy().mean()

            img = cstc.as_volume(src_sample,  mri_resolution=False)
            cname = '{} {} cluster {} - {}s'.format(p,c,v+1,np.round(ctime,2))
            print(cname)
            cfig = plt.figure()
            brain = plot_stat_map(img, subs_dir + 'fsaverage/mri/T1.mgz',draw_cross=False,black_bg=True,
                          #output_file = figures_dir + 'patterns_sources_' + cname + '.pdf',
                          vmax=1,symmetric_cbar=True,figure=cfig, threshold=.025)
            plt.show()
            
            # brain = cstc.copy().crop(tmin=ctime-.1,tmax=ctime+.1).mean().plot(subjects_dir=subs_dir,#size=1600,
            #                                                              clim={'kind': 'value', 'pos_lims': [0.001,.2,1.2]},#time_viewer=True,
            #                      subject='fsaverage', mode = 'stat_map',src=src_sample, initial_time = ctime)#,)
 

In [ ]:
from nilearn import datasets
from nilearn import surface
from nilearn import plotting
fsaverage = datasets.fetch_surf_fsaverage()
fsaverage

In [ ]:
#conds = ['maintenance1','manipulation1','interaction']#'maintenance1','manipulation1',]
#csfname = stats[].get_keys()
views = ['lateral','medial']
hemi = ['left','right']
surfs = [fsaverage.pial_left, fsaverage.pial_right]
surfs2 = [fsaverage.infl_left, fsaverage.infl_right]
bgs = [fsaverage.sulc_left, fsaverage.sulc_right]
for p in stats:
    conds = stats[p].keys()
    for c in conds:#stats[p]:   
        vx, vy, masks, pvals = get_cluster_peaks(stats[p][c],p_threshold=.1)
        print(vy)
        for v in range(vy.shape[0]):
            cstc = stc.copy().crop(tmin=periods[p][0],tmax=periods[p][1])
            cstc.data = stats[p][c]['data_mean']*masks[v]
            ctime = cstc.times[vy[v]].copy()

            #cstc = cstc.copy().crop(tmin=ctime-.1,tmax=ctime+.1).mean()
            cstc = cstc.copy().mean()
            img = cstc.as_volume(src_sample,  mri_resolution=False)
            
            cname = '{} {} cluster {} - {}s (p = {})'.format(p,c,v+1,np.round(ctime,2),np.round(pvals[v],3))
            print(cname)
            cfig, caxis = plt.subplots(nrows=2,ncols=2,subplot_kw={'projection': '3d'})
            for vix, vi in enumerate(views):
                for hix,h in enumerate(hemi):
                    surf = surface.vol_to_surf(img, surfs[hix])
                    plotting.plot_surf_stat_map(surfs2[hix],surf, hemi=h,axes=caxis[vix,hix],
                                                   symmetric_cbar=True,view=vi, vmax=1,threshold=.025,
                                                    colorbar=True, bg_map=bgs[hix],darkness=1)#,#,
                                                   # engine='matplotlib',bg_on_data=True)
            plt.show()
            #cfig.savefig(figures_dir + 'patterns_sources_' + cname + '_surface.pdf')
            # brain = cstc.copy().crop(tmin=ctime-.1,tmax=ctime+.1).mean().plot(subjects_dir=subs_dir,#size=1600,
            #                                                              clim={'kind': 'value', 'pos_lims': [0.001,.2,1.2]},#time_viewer=True,
            #                      subject='fsaverage', mode = 'stat_map',src=src_sample, initial_time = ctime)#,)
 

In [ ]:
#conds = ['maintenance1','manipulation1','interaction']#'maintenance1','manipulation1',]
views = ['lateral','medial']
hemi = ['left','right']
surfs = [fsaverage.pial_left, fsaverage.pial_right]
surfs2 = [fsaverage.infl_left, fsaverage.infl_right]
bgs = [fsaverage.sulc_left, fsaverage.sulc_right]
for p in stats:
    conds = stats[p].keys()
    for c in conds:#stats[p]:   
        # vx, vy, masks, pvals = get_cluster_peaks(stats[p][c])
        # print(vy)
    #for v in range(vy.shape[0]):
        cstc = stc.copy().crop(tmin=periods[p][0],tmax=periods[p][1])
        cstc.data = stats[p][c]['data_mean']*stats[p][c]['mask']
        #ctime = cstc.times[vy[v]].copy()

        #cstc = cstc.copy().crop(tmin=ctime-.1,tmax=ctime+.1).mean()
        cstc = cstc.copy().mean()
        img = cstc.as_volume(src_sample,  mri_resolution=False)
        
        cname = '{} {}'.format(p,c)
        print(cname)
        cfig, caxis = plt.subplots(nrows=2,ncols=2,subplot_kw={'projection': '3d'})
        for vix, vi in enumerate(views):
            for hix,h in enumerate(hemi):
                surf = surface.vol_to_surf(img, surfs[hix])
                plotting.plot_surf_stat_map(surfs2[hix],surf, hemi=h,axes=caxis[vix,hix],
                                                symmetric_cbar=True,view=vi, vmax=1,threshold=.025,
                                                colorbar=True, bg_map=bgs[hix],darkness=1)#,#,
                caxis[vix,hix].set_rasterized(True) #To save more efficiently

                                                # engine='matplotlib',bg_on_data=True)
        plt.show()
        cfig.savefig(figures_dir + 'patterns_sources_' + cname + '_all_clust_surface.pdf',dpi=300)
            # brain = cstc.copy().crop(tmin=ctime-.1,tmax=ctime+.1).mean().plot(subjects_dir=subs_dir,#size=1600,
            #                                                              clim={'kind': 'value', 'pos_lims': [0.001,.2,1.2]},#time_viewer=True,
            #                      subject='fsaverage', mode = 'stat_map',src=src_sample, initial_time = ctime)#,)
 

In [ ]:
conds = ['interaction']#'maintenance1','manipulation1',]
views = ['lateral','medial']
hemi = ['left','right']
surfs = [fsaverage.pial_left, fsaverage.pial_right]
surfs2 = [fsaverage.infl_left, fsaverage.infl_right]
bgs = [fsaverage.sulc_left, fsaverage.sulc_right]
for p in ['I2']:
    for c in conds:#stats[p]:   
        # vx, vy, masks, pvals = get_cluster_peaks(stats[p][c])
        # print(vy)
    #for v in range(vy.shape[0]):
        cstc = stc.copy().crop(tmin=periods[p][0],tmax=periods[p][1])
        cstc.data = stats[p][c]['data_mean']*(np.abs(stats[p][c]['tvals']) > 2)
        #ctime = cstc.times[vy[v]].copy()

        #cstc = cstc.copy().crop(tmin=ctime-.1,tmax=ctime+.1).mean()
        cstc = cstc.copy().mean()
        img = cstc.as_volume(src_sample,  mri_resolution=False)
        
        cname = '{} {}'.format(p,c)
        print(cname)
        cfig, caxis = plt.subplots(nrows=2,ncols=2,subplot_kw={'projection': '3d'})
        for vix, vi in enumerate(views):
            for hix,h in enumerate(hemi):
                surf = surface.vol_to_surf(img, surfs[hix])
                plotting.plot_surf_stat_map(surfs2[hix],surf, hemi=h,axes=caxis[vix,hix],
                                                symmetric_cbar=True,view=vi, vmax=1,threshold=.025,
                                                colorbar=True, bg_map=bgs[hix],darkness=1)#,#,
                                                # engine='matplotlib',bg_on_data=True)
                caxis[vix,hix].set_rasterized(True) #To save more efficiently

                                                # engine='matplotlib',bg_on_data=True)
        plt.show()
        cfig.savefig(figures_dir + 'patterns_sources_' + cname + '_thresholded_surface.pdf',dpi=300)
        #cfig.savefig(figures_dir + 'patterns_sources_' + cname + '_all_clust_surface.pdf')
            # brain = cstc.copy().crop(tmin=ctime-.1,tmax=ctime+.1).mean().plot(subjects_dir=subs_dir,#size=1600,
            #                                                              clim={'kind': 'value', 'pos_lims': [0.001,.2,1.2]},#time_viewer=True,
            #                      subject='fsaverage', mode = 'stat_map',src=src_sample, initial_time = ctime)#,)
 